In [1]:
# import packages
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import time

from tensorflow.keras import datasets, layers, models

2024-03-27 15:23:21.486325: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Datasets
train_paths = [
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_AREA/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_BICUBIC/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_BILINEAR/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_GAUSSIAN/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS3/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS5/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_MITCHELLCUBIC/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_NEAREST_NEIGHBOR/train'   
]
test_paths = [
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_AREA/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_BICUBIC/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_BILINEAR/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_GAUSSIAN/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS3/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS5/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_MITCHELLCUBIC/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_NEAREST_NEIGHBOR/test'   
]
csv_names = [
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/resize_methods/AREA.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/resize_methods/BICUBIC.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/resize_methods/BILINEAR.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/resize_methods/GAUSSIAN.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/resize_methods/LANCZOS3.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/resize_methods/LANCZOS5.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/resize_methods/MITCHELLCUBIC.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/resize_methods/NEAREST_NEIGHBOR.csv'
]

#train = tf.data.Dataset.load('/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16/train')
#test  = tf.data.Dataset.load('/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16/test')

def preprocess(example):
    image = example['image']
    image.set_shape([16, 16, 1])
    label = example['label']
    label = tf.one_hot(label, depth=100)  # One-hot encode the labels
    return image, label

def format_set(train_set, test_set):
    # format and cache
    train_set_formatted = train_set.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    train_set_formatted = train_set_formatted.cache()
    train_set_formatted = train_set_formatted.batch(512)
    train_set_formatted = train_set_formatted.prefetch(tf.data.AUTOTUNE)
    test_set_formatted = test_set.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    test_set_formatted = test_set_formatted.batch(512)
    test_set_formatted = test_set_formatted.cache()
    test_set_formatted = test_set_formatted.prefetch(tf.data.AUTOTUNE)
    return train_set_formatted, test_set_formatted

In [3]:
'''
# Get the first batch of images and labels from the training dataset
images, labels = next(iter(train))

# Create a figure
plt.figure(figsize=(10,10))

# Plot 25 images
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # Reshape the image
    image = np.reshape(images[i], (16, 16))
    plt.imshow(image, cmap=plt.cm.binary)
    # Get the label for the image
    label = np.argmax(labels[i])
    plt.xlabel(label)
plt.show()
'''

'\n# Get the first batch of images and labels from the training dataset\nimages, labels = next(iter(train))\n\n# Create a figure\nplt.figure(figsize=(10,10))\n\n# Plot 25 images\nfor i in range(25):\n    plt.subplot(5,5,i+1)\n    plt.xticks([])\n    plt.yticks([])\n    plt.grid(False)\n    # Reshape the image\n    image = np.reshape(images[i], (16, 16))\n    plt.imshow(image, cmap=plt.cm.binary)\n    # Get the label for the image\n    label = np.argmax(labels[i])\n    plt.xlabel(label)\nplt.show()\n'

In [4]:
# Keeping time
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()

In [5]:
# Model taken from example (https://www.tensorflow.org/tutorials/images/cnn)
def create_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (2, 2), activation='relu')) # input_shape removed, (3, 3) -> (2, 2)
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (2, 2), activation='relu')) # (3, 3) -> (2, 2)
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (2, 2), activation='relu')) # (3, 3) -> (2, 2)
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(100))  # changed from 10 to 100, due to amount of classes
    return model
    
def compile_model(model):
    model.compile(
        optimizer='adam',
        # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=['accuracy']
    )
    return model

In [6]:
#train = tf.data.Dataset.load('/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16/train')
#test  = tf.data.Dataset.load('/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16/test')

for i in range(len(csv_names)):
    # fetch datasets
    train = tf.data.Dataset.load(train_paths[i])
    test  = tf.data.Dataset.load(test_paths[i])
    train, test = format_set(train, test)

    # create model
    model = create_model()
    model = compile_model(model)

    # Train
    history = model.fit(train, epochs=2, validation_data=test, callbacks=[time_callback])

    # Convert the history.history dict to a pandas DataFrame
    hist_df = pd.DataFrame(history.history)

    # Add epoch times
    hist_df['time'] = time_callback.times

    # Save to csv
    hist_df.to_csv(csv_names[i])

Epoch 1/2
98/98 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.0116 - loss: 0.0100 - val_accuracy: 0.0244 - val_loss: 0.0099
Epoch 2/2
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.0283 - loss: 0.0099 - val_accuracy: 0.0516 - val_loss: 0.0099
Epoch 1/2
98/98 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.0109 - loss: 0.0101 - val_accuracy: 0.0150 - val_loss: 0.0099
Epoch 2/2
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.0238 - loss: 0.0099 - val_accuracy: 0.0423 - val_loss: 0.0099
Epoch 1/2
98/98 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.0124 - loss: 0.0100 - val_accuracy: 0.0282 - val_loss: 0.0099
Epoch 2/2
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0319 - loss: 0.0099 - val_accuracy: 0.0492 - val_loss: 0.0098
Epoch 1/2
98/98 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.0116 - loss: 0.0100 - val_accuracy: 0.0211 - val_loss: 0.0099
Epoch 2/2
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0326 - loss: 0.0099 - val_accuracy: 0.0490 - val_loss: